Use web scraping to pull the inv from 5 Local Dealers all of the same Make.(Keffer, Black, LakeNorman, Hendrick, and Stateline) From the data I get from the web scraping I want to answer four question:
    1- Does the inventory match proportionally between these dealerships of the different vehicles from their makes?
    2- How does the Amount of New Cars compare to that of used cars?
    3- Whats the most popular make of the used car selections?
    4- Whats the most popular on brand car?
    

page 1 https://www.kefferjeep.com/new-vehicles/?_dFR%5Blightning.isPolice%5D%5B0%5D=No
page 2 https://www.kefferjeep.com/new-vehicles/?_p=1&_dFR%5Blightning.isPolice%5D%5B0%5D=No
page 3 https://www.kefferjeep.com/new-vehicles/?_p=2&_dFR%5Blightning.isPolice%5D%5B0%5D=No

lkn
page 1 https://www.lakenormanchrysler.com/new-vehicles/?
page 2 https://www.lakenormanchrysler.com/new-vehicles/?_p=1
page 3 https://www.lakenormanchrysler.com/new-vehicles/?_p=2

In [1]:
!pip install selenium
!pip install webdriver-manager
!pip install undetected-chromedriver

In [42]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from selenium.common.exceptions import InvalidSessionIdException, WebDriverException
import undetected_chromedriver as uc

In [20]:

# Set up Chrome options
options = uc.ChromeOptions()

options.add_argument("--window-size=1920,1080")

# Automatically download the correct driver
service = Service(ChromeDriverManager().install())


In [14]:
driver = uc.Chrome(options=options)
vehicles = []
page = 0

while True:
    print(f"Scraping page {page+1}...")
    url = f"https://www.lakenormanchrysler.com/new-vehicles/?_p={page}"
    driver.get(url)
    # Wait until vehicle cards are present or timeout after 20 seconds
    timeout = 20
    poll_interval = 1
    elapsed = 0

    while elapsed < timeout:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        vehicle_cards = soup.find_all("div", class_="hit-content")
        if vehicle_cards:
            break
        time.sleep(poll_interval)
        elapsed += poll_interval
    else:
        print(f"No vehicle listings found or timed out on page {page}. Ending.")
        break
    # Parse page
    soup = BeautifulSoup(driver.page_source, "html.parser")
    vehicle_cards = soup.find_all("div", class_="hit-content")
    print(f"Found {len(vehicle_cards)} vehicle cards on page {page}")
    
    # If no vehicles are found, break the loop
    if not vehicle_cards:
        print("No more vehicles found. Done.")
        break

    for card in vehicle_cards:
        try:
            # Extract elements (not strings yet)
            condition_year_el = card.find("span", class_="title-top")
            make_model_el = card.find("span", class_="title-bottom")
            price_el = card.find("span", class_="price")
            vin_el = card.find("div", class_="vin-row")

            # Now safely extract text only if the element exists
            condition_year = condition_year_el.get_text(strip=True) if condition_year_el else None
            make_model = make_model_el.get_text(strip=True) if make_model_el else None
            price = price_el.get_text(strip=True) if price_el else None
            vin = vin_el.get_text(strip=True) if vin_el else None

            vehicles.append({
                "Condition/Year": condition_year,
                "Make-Model": make_model,
                "Price": price,
                "Vin": vin
            })
        except Exception as e:
            print("Error parsing a card:", e)

    page += 1  # Go to next page

# Quit browser and save data
driver.quit()
df = pd.DataFrame(vehicles)
df.to_csv("selenium_LKN_paginated_scrape.csv", index=False)
print(f"Scraped {len(df)} vehicles across {page} pages.")

Scraping page 1...
Found 20 vehicle cards on page 0
Scraping page 2...
Found 20 vehicle cards on page 1
Scraping page 3...
Found 20 vehicle cards on page 2
Scraping page 4...
Found 20 vehicle cards on page 3
Scraping page 5...
Found 20 vehicle cards on page 4
Scraping page 6...
Found 20 vehicle cards on page 5
Scraping page 7...
Found 20 vehicle cards on page 6
Scraping page 8...
Found 20 vehicle cards on page 7
Scraping page 9...
Found 20 vehicle cards on page 8
Scraping page 10...
Found 20 vehicle cards on page 9
Scraping page 11...
Found 20 vehicle cards on page 10
Scraping page 12...
Found 20 vehicle cards on page 11
Scraping page 13...
Found 20 vehicle cards on page 12
Scraping page 14...
Found 20 vehicle cards on page 13
Scraping page 15...
Found 20 vehicle cards on page 14
Scraping page 16...
Found 20 vehicle cards on page 15
Scraping page 17...
Found 20 vehicle cards on page 16
Scraping page 18...
Found 20 vehicle cards on page 17
Scraping page 19...
Found 20 vehicle cards on p

In [17]:
df.count()


Condition/Year    1572
Make-Model        1572
Price             1534
Vin               1572
dtype: int64

In [21]:

# Recreate ChromeOptions to avoid reuse error
options = uc.ChromeOptions()
options.add_argument("--window-size=1920,1080")

# Now this won't throw an error
driver = uc.Chrome(options=options)
vehicles = []
page = 0

while True:
    print(f"Scraping page {page+1}...")
    url = f"https://www.kefferjeep.com/new-vehicles/?_p={page}&_dFR%5Blightning.isPolice%5D%5B0%5D=No"
    driver.get(url)
    # Wait until vehicle cards are present or timeout after 20 seconds
    timeout = 20
    poll_interval = 1
    elapsed = 0

    while elapsed < timeout:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        vehicle_cards = soup.find_all("div", class_="hit-content")
        if vehicle_cards:
            break
        time.sleep(poll_interval)
        elapsed += poll_interval
    else:
        print(f"No vehicle listings found or timed out on page {page}. Ending.")
        break
    # Parse page
    soup = BeautifulSoup(driver.page_source, "html.parser")
    vehicle_cards = soup.find_all("div", class_="hit-content")
    print(f"Found {len(vehicle_cards)} vehicle cards on page {page}")
    
    # If no vehicles are found, break the loop
    if not vehicle_cards:
        print("No more vehicles found. Done.")
        break

    for card in vehicle_cards:
        try:
            # Extract elements (not strings yet)
            condition_year_el = card.find("span", class_="title-top")
            make_model_el = card.find("span", class_="title-bottom")
            price_el = card.find("span", class_="price")
            vin_el = card.find("div", class_="vin-row")

            # Now safely extract text only if the element exists
            condition_year = condition_year_el.get_text(strip=True) if condition_year_el else None
            make_model = make_model_el.get_text(strip=True) if make_model_el else None
            price = price_el.get_text(strip=True) if price_el else None
            vin = vin_el.get_text(strip=True) if vin_el else None

            vehicles.append({
                "Condition/Year": condition_year,
                "Make-Model": make_model,
                "Price": price,
                "Vin": vin
            })
        except Exception as e:
            print("Error parsing a card:", e)

    page += 1  # Go to next page

# Quit browser and save data
driver.quit()
df = pd.DataFrame(vehicles)
df.to_csv("selenium_kef_paginated_scrape.csv", index=False)
print(f"Scraped {len(df)} vehicles across {page} pages.")

Scraping page 1...
Found 20 vehicle cards on page 0
Scraping page 2...
Found 20 vehicle cards on page 1
Scraping page 3...
Found 20 vehicle cards on page 2
Scraping page 4...
Found 20 vehicle cards on page 3
Scraping page 5...
Found 20 vehicle cards on page 4
Scraping page 6...
Found 20 vehicle cards on page 5
Scraping page 7...
Found 20 vehicle cards on page 6
Scraping page 8...
Found 20 vehicle cards on page 7
Scraping page 9...
Found 20 vehicle cards on page 8
Scraping page 10...
Found 20 vehicle cards on page 9
Scraping page 11...
Found 20 vehicle cards on page 10
Scraping page 12...
Found 20 vehicle cards on page 11
Scraping page 13...
Found 20 vehicle cards on page 12
Scraping page 14...
Found 20 vehicle cards on page 13
Scraping page 15...
Found 20 vehicle cards on page 14
Scraping page 16...
Found 20 vehicle cards on page 15
Scraping page 17...
Found 20 vehicle cards on page 16
Scraping page 18...
Found 20 vehicle cards on page 17
Scraping page 19...
Found 20 vehicle cards on p

In [22]:
df.count()


Condition/Year    579
Make-Model        579
Price             572
Vin               579
dtype: int64

In [29]:
# Recreate ChromeOptions to avoid reuse error
options = uc.ChromeOptions()
options.add_argument("--window-size=1920,1080")

# Now this won't throw an error
driver = uc.Chrome(options=options)
vehicles = []
page = 0

while True:
    print(f"Scraping page {page+1}...")
    url = f"https://www.statelinechryslerjeepdodge.net/new-vehicles/?_p={page}&_dFR%5Blightning.isPolice%5D%5B0%5D=No"
    driver.get(url)
    # Wait until vehicle cards are present or timeout after 20 seconds
    timeout = 20
    poll_interval = 1
    elapsed = 0

    while elapsed < timeout:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        vehicle_cards = soup.find_all("div", class_="hit-content")
        if vehicle_cards:
            break
        time.sleep(poll_interval)
        elapsed += poll_interval
    else:
        print(f"No vehicle listings found or timed out on page {page}. Ending.")
        break
    # Parse page
    soup = BeautifulSoup(driver.page_source, "html.parser")
    vehicle_cards = soup.find_all("div", class_="hit-content")
    print(f"Found {len(vehicle_cards)} vehicle cards on page {page}")
    
    # If no vehicles are found, break the loop
    if not vehicle_cards:
        print("No more vehicles found. Done.")
        break

    for card in vehicle_cards:
        try:
            # Extract elements (not strings yet)
            condition_year_el = card.find("span", class_="title-top")
            make_model_el = card.find("span", class_="title-bottom")
            price_el = card.find("span", class_="price")
            vin_el = card.find("div", class_="vin-row")

            # Now safely extract text only if the element exists
            condition_year = condition_year_el.get_text(strip=True) if condition_year_el else None
            make_model = make_model_el.get_text(strip=True) if make_model_el else None
            price = price_el.get_text(strip=True) if price_el else None
            vin = vin_el.get_text(strip=True) if vin_el else None

            vehicles.append({
                "Condition/Year": condition_year,
                "Make-Model": make_model,
                "Price": price,
                "Vin": vin
            })
        except Exception as e:
            print("Error parsing a card:", e)

    page += 1  # Go to next page

# Quit browser and save data
driver.quit()
df = pd.DataFrame(vehicles)
df.to_csv("selenium_stateline_paginated_scrape.csv", index=False)
print(f"Scraped {len(df)} vehicles across {page} pages.")

Scraping page 1...


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=135.0.7049.85)
Stacktrace:
	GetHandleVerifier [0x00B48073+60707]
	GetHandleVerifier [0x00B480B4+60772]
	(No symbol) [0x00970683]
	(No symbol) [0x0094FBEE]
	(No symbol) [0x009E3C2E]
	(No symbol) [0x009FE129]
	(No symbol) [0x009DCE46]
	(No symbol) [0x009AC5D3]
	(No symbol) [0x009AD424]
	GetHandleVerifier [0x00D8BB53+2435075]
	GetHandleVerifier [0x00D870F3+2416035]
	GetHandleVerifier [0x00DA349C+2531660]
	GetHandleVerifier [0x00B5F145+155125]
	GetHandleVerifier [0x00B65AED+182173]
	GetHandleVerifier [0x00B4F948+91640]
	GetHandleVerifier [0x00B4FAF0+92064]
	GetHandleVerifier [0x00B3A5B0+4704]
	BaseThreadInitThunk [0x76645D49+25]
	RtlInitializeExceptionChain [0x777CCF0B+107]
	RtlGetAppContainerNamedObjectPath [0x777CCE91+561]


In [26]:
df.count()


Condition/Year    2346
Make-Model        2346
Price             2295
Vin                  0
dtype: int64

In [45]:
def start_driver():
    options = uc.ChromeOptions()
    options.add_argument("--window-size=1920,1080")
    return uc.Chrome(options=options)

vehicles = []
entry = 0
max_retries = 3

driver = start_driver()

while True:
    retries = 0
    success = False

    while retries < max_retries:
        try:
            print(f"Scraping page {entry + 1} (attempt {retries + 1})...")
            url = f"https://www.hendrickchryslerdodgejeepramofconcord.com/new-inventory/index.htm?start={entry}"
            driver.get(url)

            timeout = 20
            poll_interval = 1
            elapsed = 0

            while elapsed < timeout:
                soup = BeautifulSoup(driver.page_source, "html.parser")
                vehicle_cards = soup.find_all("div", class_="vehicle-card-details-container")
                if vehicle_cards:
                    break
                time.sleep(poll_interval)
                elapsed += poll_interval
            else:
                print(f"No vehicle listings found or timed out on page {entry}. Ending.")
                success = True  # Treat as success to exit retry loop, but end scrape
                break

            # Process vehicle cards
            print(f"Found {len(vehicle_cards)} vehicle cards on page {entry}")
            if not vehicle_cards:
                print("No more vehicles found. Done.")
                success = True
                break

            for card in vehicle_cards:
                try:
                    condition_year_make_model = card.find("h2", class_="vehicle-card-title mt-0 d-block mb-0 justify-content-between align-items-end h5 inv-type-new")
                    price = card.find("span", class_="price-value")

                    vehicles.append({
                        "Condition/Year/make/model": condition_year_make_model.get_text(strip=True) if condition_year else None,
                        "Price": price.get_text(strip=True) if price else None,
                    })
                except Exception as e:
                    print("Error parsing a card:", e)

            success = True
            break  # Exit retry loop

        except (InvalidSessionIdException, WebDriverException) as e:
            print(f"Error on page {page}: {e}")
            try:
                driver.quit()
            except:
                pass
            print("Restarting browser...")
            driver = start_driver()
            retries += 1
            time.sleep(3)

    if not success:
        print(f"Skipping page {page} after {max_retries} retries.")
        page += 1
        continue

    if not vehicle_cards:
        break  # No more vehicles; end scraping

    entry += 18

# Quit browser and save data
try:
    driver.quit()
except:
    pass

df = pd.DataFrame(vehicles)
df.to_csv("selenium_hend_paginated_scrape.csv", index=False)
print(f"Scraped {len(df)} vehicles across {page} pages.")

Scraping page 1 (attempt 1)...
Found 18 vehicle cards on page 0
Scraping page 19 (attempt 1)...
Found 18 vehicle cards on page 18
Scraping page 37 (attempt 1)...
Found 18 vehicle cards on page 36
Scraping page 55 (attempt 1)...
Found 18 vehicle cards on page 54
Scraping page 73 (attempt 1)...
Found 18 vehicle cards on page 72
Scraping page 91 (attempt 1)...
Found 18 vehicle cards on page 90
Scraping page 109 (attempt 1)...
Found 18 vehicle cards on page 108
Scraping page 127 (attempt 1)...
Found 18 vehicle cards on page 126
Scraping page 145 (attempt 1)...
Found 18 vehicle cards on page 144
Scraping page 163 (attempt 1)...
Found 18 vehicle cards on page 162
Scraping page 181 (attempt 1)...
Found 18 vehicle cards on page 180
Scraping page 199 (attempt 1)...
Found 18 vehicle cards on page 198
Scraping page 217 (attempt 1)...
Found 18 vehicle cards on page 216
Scraping page 235 (attempt 1)...
Found 18 vehicle cards on page 234
Scraping page 253 (attempt 1)...
Found 18 vehicle cards on pag

In [46]:
df.count()

Condition/Year/make/model    1637
Price                         273
dtype: int64